# Project 2 - Lab 5 - Filter and aggregate the water quality data

Recall that one of the files (starts with `mces`) contains water quality measurements for lakes in the Twin Cities.  In this lab, we will narrow down the list of lakes for which we have at least one of each measurement type (phosphorus and secchi depth) for each year between 2004 and 2015.

## Problem 1 - Inspect the data

We will be focusing on two of water quality measurements: phosphorus and secchi depth.  Before we start trimming the data set, we should explore these metrics.

1. Each of the measures has a `QUALIFIER` column.  Group and aggregate by each of these columns and note any problematic values.  **Hint.** This search should indicate that some of the phosphorus measurements should be dropped.  Make sure you include this in your primary query.
2. Each measure also includes a `Units` column.  Check that all measurement are in the same units, and convert as needed.

In [2]:
# Your code here
from pyspark.sql.functions import *
from pyspark.sql import SparkSession
from more_pyspark import *

pd.set_option('display.max_columns', None)

spark = SparkSession.builder.appName('Ops').getOrCreate()

22/12/01 12:09:49 WARN Utils: Your hostname, jt7372wd222 resolves to a loopback address: 127.0.1.1; using 172.19.119.45 instead (on interface eth0)
22/12/01 12:09:49 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/12/01 12:09:50 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
water_quality = spark.read.csv('data/MinneMUDAC_raw_files/mces_lakes_1999_2014.txt', sep='\t', header=True)

In [4]:
water_quality.take(5)>>to_pandas

22/12/01 12:09:55 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


,PROJECT_ID,DATA_SET_TITLE,LAKE_NAME,CITY,COUNTY,DNR_ID_Site_Number,MAJOR_WATERSHED,WATER_PLANNING_AUTHORITY,LAKE_SITE_NUMBER,START_DATE,START_HOURMIN24,END_DATE,END_HOURMIN24,SAMPLE_DEPTH_IN_METERS,Seasonal_Lake_Grade_RESULT,Seasonal_Lake_Grade_QUALIFIER,Seasonal_Lake_Grade_Units,Physical_Condition_RESULT,Physical_Condition_QUALIFIER,Physical_Condition_Units,Recreational_Suitability_RESULT,Recreational_Suitability_QUALIFIER,Recreational_Suitability_Units,Secchi_Depth_RESULT_SIGN,Secchi_Depth_RESULT,Secchi_Depth_QUALIFIER,Secchi_Depth_Units,Total_Phosphorus_RESULT_SIGN,Total_Phosphorus_RESULT,Total_Phosphorus_QUALIFIER,Total_Phosphorus_Units,longitude,latitude
0,7108,Citizen Assisted Monitoring Program (CAMP) for...,Acorn Lake,Oakdale,Washington,82010200-01,Lower St. Croix River,Valley Branch WD,1,2006-04-16,0:00,2006-04-16,0:00,0,None,None,0-4 Categorical Calculated Seasonally: 4 good ...,1,Approved,1-5 Categorical: 1 good & 5 bad,5,Approved,1-5 Categorical: 1 good & 5 bad,None,1,Approved,m,None,0.156,Approved,mg/L,-92.97171054,45.01655642
1,7108,Citizen Assisted Monitoring Program (CAMP) for...,Acorn Lake,Oakdale,Washington,82010200-01,Lower St. Croix River,Valley Branch WD,1,2006-05-01,0:00,2006-09-30,0:00,0,2,Approved,0-4 Categorical Calculated Seasonally: 4 good ...,None,None,1-5 Categorical: 1 good & 5 bad,None,None,1-5 Categorical: 1 good & 5 bad,None,None,None,m,None,None,None,mg/L,-92.97171054,45.01655642
2,7108,Citizen Assisted Monitoring Program (CAMP) for...,Acorn Lake,Oakdale,Washington,82010200-01,Lower St. Croix River,Valley Branch WD,1,2006-05-02,0:00,2006-05-02,0:00,0,None,None,0-4 Categorical Calculated Seasonally: 4 good ...,1,Approved,1-5 Categorical: 1 good & 5 bad,5,Approved,1-5 Categorical: 1 good & 5 bad,None,0.66,Approved,m,None,0.107,Approved,mg/L,-92.97171054,45.01655642
3,7108,Citizen Assisted Monitoring Program (CAMP) for...,Acorn Lake,Oakdale,Washington,82010200-01,Lower St. Croix River,Valley Branch WD,1,2006-05-16,0:00,2006-05-16,0:00,0,None,None,0-4 Categorical Calculated Seasonally: 4 good ...,2,Approved,1-5 Categorical: 1 good & 5 bad,5,Approved,1-5 Categorical: 1 good & 5 bad,None,0.66,Approved,m,None,0.141,Approved,mg/L,-92.97171054,45.01655642
4,7108,Citizen Assisted Monitoring Program (CAMP) for...,Acorn Lake,Oakdale,Washington,82010200-01,Lower St. Croix River,Valley Branch WD,1,2006-05-30,0:00,2006-05-30,0:00,0,None,None,0-4 Categorical Calculated Seasonally: 4 good ...,2,Approved,1-5 Categorical: 1 good & 5 bad,5,Approved,1-5 Categorical: 1 good & 5 bad,None,0.5,Approved,m,None,0.029,Approved,mg/L,-92.97171054,45.01655642


In [5]:
water_quality.groupBy('Secchi_Depth_QUALIFIER').count().collect() >> to_pandas

,Secchi_Depth_QUALIFIER,count
0,Approved,35104
1,None,13153


In [6]:
water_quality.groupBy('Total_Phosphorus_QUALIFIER').count().collect() >> to_pandas

,Total_Phosphorus_QUALIFIER,count
0,Approved,43639
1,None,4583
2,Suspect,35


In [7]:
water_quality.groupBy('Secchi_Depth_Units').count().collect() >> to_pandas


,Secchi_Depth_Units,count
0,m,48257


In [8]:
water_quality.groupBy('Total_Phosphorus_Units').count().collect() >> to_pandas

,Total_Phosphorus_Units,count
0,mg/L,48257


## Problem 2 - Find filter and aggregate.

#### Tasks

Build a query that groups and aggregates to find the yearly average values for both phosphorus and secchi depth.  To do this your will want to

1. Filter based on what you learned in **Problem 1.**
2. Make sure that the `END_DATE` has the correct type and extract the year.  
3. Filter to the correct range of years.
4. Now you should group and aggregate to compute the yearly means.  We want to keep both the `LAKE_NAME` and lake ID.  To do this you should group by both, as well as by year.

In [9]:
# Your code here
water_quality_filtered = (water_quality
 .where((col('Secchi_Depth_QUALIFIER') == 'Approved') & (col('Total_Phosphorus_QUALIFIER') == 'Approved'))
)

In [10]:
water_quality_filtered.take(2) >> to_pandas

,PROJECT_ID,DATA_SET_TITLE,LAKE_NAME,CITY,COUNTY,DNR_ID_Site_Number,MAJOR_WATERSHED,WATER_PLANNING_AUTHORITY,LAKE_SITE_NUMBER,START_DATE,START_HOURMIN24,END_DATE,END_HOURMIN24,SAMPLE_DEPTH_IN_METERS,Seasonal_Lake_Grade_RESULT,Seasonal_Lake_Grade_QUALIFIER,Seasonal_Lake_Grade_Units,Physical_Condition_RESULT,Physical_Condition_QUALIFIER,Physical_Condition_Units,Recreational_Suitability_RESULT,Recreational_Suitability_QUALIFIER,Recreational_Suitability_Units,Secchi_Depth_RESULT_SIGN,Secchi_Depth_RESULT,Secchi_Depth_QUALIFIER,Secchi_Depth_Units,Total_Phosphorus_RESULT_SIGN,Total_Phosphorus_RESULT,Total_Phosphorus_QUALIFIER,Total_Phosphorus_Units,longitude,latitude
0,7108,Citizen Assisted Monitoring Program (CAMP) for...,Acorn Lake,Oakdale,Washington,82010200-01,Lower St. Croix River,Valley Branch WD,1,2006-04-16,0:00,2006-04-16,0:00,0,None,None,0-4 Categorical Calculated Seasonally: 4 good ...,1,Approved,1-5 Categorical: 1 good & 5 bad,5,Approved,1-5 Categorical: 1 good & 5 bad,None,1,Approved,m,None,0.156,Approved,mg/L,-92.97171054,45.01655642
1,7108,Citizen Assisted Monitoring Program (CAMP) for...,Acorn Lake,Oakdale,Washington,82010200-01,Lower St. Croix River,Valley Branch WD,1,2006-05-02,0:00,2006-05-02,0:00,0,None,None,0-4 Categorical Calculated Seasonally: 4 good ...,1,Approved,1-5 Categorical: 1 good & 5 bad,5,Approved,1-5 Categorical: 1 good & 5 bad,None,0.66,Approved,m,None,0.107,Approved,mg/L,-92.97171054,45.01655642


In [11]:
print(water_quality_filtered.schema["END_DATE"].dataType)

StringType()


In [12]:
water_quality_year_added = (water_quality_filtered
 .withColumn('Year', year(col('END_DATE')))
)

In [13]:
water_quality_year_added.take(2) >> to_pandas

,PROJECT_ID,DATA_SET_TITLE,LAKE_NAME,CITY,COUNTY,DNR_ID_Site_Number,MAJOR_WATERSHED,WATER_PLANNING_AUTHORITY,LAKE_SITE_NUMBER,START_DATE,START_HOURMIN24,END_DATE,END_HOURMIN24,SAMPLE_DEPTH_IN_METERS,Seasonal_Lake_Grade_RESULT,Seasonal_Lake_Grade_QUALIFIER,Seasonal_Lake_Grade_Units,Physical_Condition_RESULT,Physical_Condition_QUALIFIER,Physical_Condition_Units,Recreational_Suitability_RESULT,Recreational_Suitability_QUALIFIER,Recreational_Suitability_Units,Secchi_Depth_RESULT_SIGN,Secchi_Depth_RESULT,Secchi_Depth_QUALIFIER,Secchi_Depth_Units,Total_Phosphorus_RESULT_SIGN,Total_Phosphorus_RESULT,Total_Phosphorus_QUALIFIER,Total_Phosphorus_Units,longitude,latitude,Year
0,7108,Citizen Assisted Monitoring Program (CAMP) for...,Acorn Lake,Oakdale,Washington,82010200-01,Lower St. Croix River,Valley Branch WD,1,2006-04-16,0:00,2006-04-16,0:00,0,None,None,0-4 Categorical Calculated Seasonally: 4 good ...,1,Approved,1-5 Categorical: 1 good & 5 bad,5,Approved,1-5 Categorical: 1 good & 5 bad,None,1,Approved,m,None,0.156,Approved,mg/L,-92.97171054,45.01655642,2006
1,7108,Citizen Assisted Monitoring Program (CAMP) for...,Acorn Lake,Oakdale,Washington,82010200-01,Lower St. Croix River,Valley Branch WD,1,2006-05-02,0:00,2006-05-02,0:00,0,None,None,0-4 Categorical Calculated Seasonally: 4 good ...,1,Approved,1-5 Categorical: 1 good & 5 bad,5,Approved,1-5 Categorical: 1 good & 5 bad,None,0.66,Approved,m,None,0.107,Approved,mg/L,-92.97171054,45.01655642,2006


In [14]:
print(water_quality_year_added.schema["Year"].dataType)

IntegerType()


In [15]:
water_quality_year_filtered = water_quality_year_added.where(col('Year') > 2003)

In [22]:
water_quality_year_filtered.groupBy('Year').count().orderBy(col('Year').asc()).show()

+----+-----+
|Year|count|
+----+-----+
|2004| 1619|
|2005| 1927|
|2006| 1966|
|2007| 1602|
|2008| 2017|
|2009| 1859|
|2010| 1963|
|2011| 1776|
|2012| 1460|
|2013| 1498|
|2014| 1914|
+----+-----+



In [16]:
water_quality_year_filtered.sample(0.01).collect() >> to_pandas

,PROJECT_ID,DATA_SET_TITLE,LAKE_NAME,CITY,COUNTY,DNR_ID_Site_Number,MAJOR_WATERSHED,WATER_PLANNING_AUTHORITY,LAKE_SITE_NUMBER,START_DATE,START_HOURMIN24,END_DATE,END_HOURMIN24,SAMPLE_DEPTH_IN_METERS,Seasonal_Lake_Grade_RESULT,Seasonal_Lake_Grade_QUALIFIER,Seasonal_Lake_Grade_Units,Physical_Condition_RESULT,Physical_Condition_QUALIFIER,Physical_Condition_Units,Recreational_Suitability_RESULT,Recreational_Suitability_QUALIFIER,Recreational_Suitability_Units,Secchi_Depth_RESULT_SIGN,Secchi_Depth_RESULT,Secchi_Depth_QUALIFIER,Secchi_Depth_Units,Total_Phosphorus_RESULT_SIGN,Total_Phosphorus_RESULT,Total_Phosphorus_QUALIFIER,Total_Phosphorus_Units,longitude,latitude,Year
0,7108,Citizen Assisted Monitoring Program (CAMP) for...,Armstrong Lake,Oakdale,Washington,82011602-01,Mississippi River - Twin Cities,South Washington WD,1,2014-10-09,15:50,2014-10-09,15:50,0,None,None,0-4 Categorical Calculated Seasonally: 4 good ...,2,Approved,1-5 Categorical: 1 good & 5 bad,4,Approved,1-5 Categorical: 1 good & 5 bad,>,1.07,Approved,m,None,0.017,Approved,mg/L,-92.93917709,44.96252306,2014
1,7108,Citizen Assisted Monitoring Program (CAMP) for...,Barker Lake,Hugo,Washington,82007600-01,Lower St. Croix River,Carnelian-Marine-St. Croix WD,1,2013-10-10,11:25,2013-10-10,11:25,0,None,None,0-4 Categorical Calculated Seasonally: 4 good ...,3,Approved,1-5 Categorical: 1 good & 5 bad,4,Approved,1-5 Categorical: 1 good & 5 bad,None,1.21,Approved,m,None,0.081,Approved,mg/L,-92.89930946,45.18460321,2013
2,7108,Citizen Assisted Monitoring Program (CAMP) for...,Bass Lake,Grant,Washington,82012300-01,Lower St. Croix River,Browns Creek WD,1,2011-10-11,9:30,2011-10-11,9:30,0,None,None,0-4 Categorical Calculated Seasonally: 4 good ...,1,Approved,1-5 Categorical: 1 good & 5 bad,1,Approved,1-5 Categorical: 1 good & 5 bad,None,4.3,Approved,m,None,0.104,Approved,mg/L,-92.91784653,45.09714182,2011
3,7108,Citizen Assisted Monitoring Program (CAMP) for...,Bavaria Lake,Chaska,Carver,10001900-01,Lower Minnesota River,Carver County WMO,1,2009-08-22,13:00,2009-08-22,13:00,0,None,None,None,2,Approved,None,3,Approved,None,None,1.5,Approved,m,None,0.079,Approved,mg/L,-93.63778927,44.83812233,2009
4,7108,Citizen Assisted Monitoring Program (CAMP) for...,Bavaria Lake,Chaska,Carver,10001900-01,Lower Minnesota River,Carver County WMO,1,2010-07-23,13:30,2010-07-23,13:30,0,None,None,None,3,Approved,None,3,Approved,None,None,1,Approved,m,None,0.025,Approved,mg/L,-93.63778927,44.83812233,2010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
206,7108,Citizen Assisted Monitoring Program (CAMP) for...,Winkler Lake,Benton Township,Carver,10006600-01,Lower Minnesota River,Carver County WMO,1,2010-08-31,9:10,2010-08-31,9:10,0,None,None,None,3,Approved,None,3,Approved,None,None,0.2,Approved,m,None,0.313,Approved,mg/L,-93.83268694,44.79006836,2010
207,7108,Citizen Assisted Monitoring Program (CAMP) for...,Wood Lake,Burnsville,Dakota,19002400-01,Lower Minnesota River,Black Dog WMO,1,2005-08-22,0:00,2005-08-22,0:00,0,None,None,None,5,Approved,None,4,Approved,None,None,0.67,Approved,m,None,0.035,Approved,mg/L,-93.2658604,44.74111827,2005
208,7108,Citizen Assisted Monitoring Program (CAMP) for...,Wood Lake,Burnsville,Dakota,19002400-01,Lower Minnesota River,Black Dog WMO,1,2009-08-10,11:45,2009-08-10,11:45,0,None,None,None,2,Approved,None,4,Approved,None,None,1.5,Approved,m,None,0.039,Approved,mg/L,-93.2658604,44.74111827,2009
209,7108,Citizen Assisted Monitoring Program (CAMP) for...,Wood Lake,Burnsville,Dakota,19002400-01,Lower Minnesota River,Black Dog WMO,1,2014-08-11,13:00,2014-08-11,13:00,0,None,None,None,4,Approved,None,3,Approved,None,None,1.2,Approved,m,None,0.037,Approved,mg/L,-93.2658604,44.74111827,2014


In [24]:
new_water_quality = (
    water_quality
    .where((col('Secchi_Depth_QUALIFIER') == 'Approved') & (col('Total_Phosphorus_QUALIFIER') == 'Approved'))
    .withColumn('Year', year(col('END_DATE')))
    .where(col('Year') > 2003)
    .groupBy('LAKE_NAME', 'DNR_ID_Site_Number', 'Year')
    .agg(avg('Secchi_Depth_RESULT').alias('Mean_Secchi_Depth_Result'),
         avg('Total_Phosphorus_RESULT').alias('Mean_Phosporus_Result'))
)

new_water_quality.collect()>>to_pandas

,LAKE_NAME,DNR_ID_Site_Number,Year,Mean_Secchi_Depth_Result,Mean_Phosporus_Result
0,Alimagnet Lake,19002100-01,2006,0.525000,0.154375
1,George Watch Lake,02000500-01,2006,0.728571,0.164286
2,Big Woods Lake,10024900-01,2014,0.475000,0.167083
3,Cedar Lake,70009100-01,2009,0.800000,0.242000
4,Alimagnet Lake,19002100-01,2014,0.966667,0.082500
...,...,...,...,...,...
1644,Riley Lake,10000200-02,2009,1.700000,0.118000
1645,Sunset Pond,19045100-01,2011,2.242857,0.045500
1646,Sylvan Lake,27017100-01,2008,1.925000,0.426500
1647,Winkler Lake,10006600-01,2008,0.694444,0.253556


## Problem 3 - Find years with complete measurements.

Build a query that leads to a list of lake names and codes that fit the following criteria.

1. Only contains years after 2003.
2. Has a none-missing value for both means.
3. Contains both the lake name and the lake code.

You should save this list of lake IDs in a variable named `lakes_w_complete_info` in a file named `lake.py`.  Restart your kernel and confirm that you can import this data.

1. We did it in previous task and checked for it.

In [28]:
# Your code here
new_water_quality.where(col('Mean_Secchi_Depth_Result').isNull()).show()

+---------+------------------+----+------------------------+---------------------+
|LAKE_NAME|DNR_ID_Site_Number|Year|Mean_Secchi_Depth_Result|Mean_Phosporus_Result|
+---------+------------------+----+------------------------+---------------------+
+---------+------------------+----+------------------------+---------------------+



In [29]:
new_water_quality.where(col('Mean_Phosporus_Result').isNull()).show()

+---------+------------------+----+------------------------+---------------------+
|LAKE_NAME|DNR_ID_Site_Number|Year|Mean_Secchi_Depth_Result|Mean_Phosporus_Result|
+---------+------------------+----+------------------------+---------------------+
+---------+------------------+----+------------------------+---------------------+



In [30]:
new_water_quality.where(col('LAKE_NAME').isNull()).show()

+---------+------------------+----+------------------------+---------------------+
|LAKE_NAME|DNR_ID_Site_Number|Year|Mean_Secchi_Depth_Result|Mean_Phosporus_Result|
+---------+------------------+----+------------------------+---------------------+
+---------+------------------+----+------------------------+---------------------+



In [31]:
new_water_quality.where(col('DNR_ID_Site_Number').isNull()).show()

+---------+------------------+----+------------------------+---------------------+
|LAKE_NAME|DNR_ID_Site_Number|Year|Mean_Secchi_Depth_Result|Mean_Phosporus_Result|
+---------+------------------+----+------------------------+---------------------+
+---------+------------------+----+------------------------+---------------------+



In [33]:
complete_lakes = (new_water_quality
 .groupBy('LAKE_NAME', 'DNR_ID_Site_Number')
 .count()
 .where(col('count') == 11)
)

In [35]:
complete_lakes.collect()>>to_pandas

,LAKE_NAME,DNR_ID_Site_Number,count
0,Big Comfort Lake,13005300-01,11
1,Bone Lake,82005400-01,11
2,Colby Lake,82009400-01,11
3,Jane Lake,82010400-01,11
4,Riley Lake,10000200-01,11
5,Regional Park Lake,82008700-01,11
6,Westwood Lake,27071100-01,11
7,Eagle Lake,10012100-01,11
8,Powers Lake,82009200-01,11
9,Klawitter Pond,82036800-01,11


In [36]:
lakes_w_complete_info = list(complete_lakes.select('DNR_ID_Site_Number').toPandas()['DNR_ID_Site_Number'])

In [37]:
lakes_w_complete_info

['13005300-01',
 '82005400-01',
 '82009400-01',
 '82010400-01',
 '10000200-01',
 '82008700-01',
 '27071100-01',
 '10012100-01',
 '82009200-01',
 '82036800-01',
 '82011602-01',
 '27007000-01',
 '19002300-01',
 '82007700-01',
 '10001900-01',
 '19002900-01',
 '82003400-01',
 '19002400-01',
 '82008900-01',
 '19003100-01',
 '10009500-01',
 '19044600-01',
 '82010100-01',
 '82015900-01',
 '82009002-01',
 '27005300-01',
 '82010300-01',
 '82013700-01',
 '82002000-01',
 '27004201-01',
 '82009700-01',
 '82012200-01',
 '19002100-01',
 '19002700-01',
 '27003501-01',
 '82011301-01',
 '02000500-01',
 '82033400-01',
 '82012300-01',
 '27062700-01',
 '19034800-01',
 '82015300-01',
 '19002200-01',
 '70002600-01',
 '10001100-01',
 '19003300-01',
 '19002500-01',
 '10005200-01',
 '19002601-01']

In [38]:
with open('lake.py', 'w') as f: 
    f.write(f'lakes_w_complete_info = {lakes_w_complete_info}')

In [39]:
from lake import lakes_w_complete_info as lake_names

lake_names

['13005300-01',
 '82005400-01',
 '82009400-01',
 '82010400-01',
 '10000200-01',
 '82008700-01',
 '27071100-01',
 '10012100-01',
 '82009200-01',
 '82036800-01',
 '82011602-01',
 '27007000-01',
 '19002300-01',
 '82007700-01',
 '10001900-01',
 '19002900-01',
 '82003400-01',
 '19002400-01',
 '82008900-01',
 '19003100-01',
 '10009500-01',
 '19044600-01',
 '82010100-01',
 '82015900-01',
 '82009002-01',
 '27005300-01',
 '82010300-01',
 '82013700-01',
 '82002000-01',
 '27004201-01',
 '82009700-01',
 '82012200-01',
 '19002100-01',
 '19002700-01',
 '27003501-01',
 '82011301-01',
 '02000500-01',
 '82033400-01',
 '82012300-01',
 '27062700-01',
 '19034800-01',
 '82015300-01',
 '19002200-01',
 '70002600-01',
 '10001100-01',
 '19003300-01',
 '19002500-01',
 '10005200-01',
 '19002601-01']

In [40]:
len(lake_names)

49

## Problem 4 - Create and write the final water quality table.

Finally, you should filter the table from **Problem 2.** to the lakes with complete information, then write this table to a `parquet` "file" named `water_quality_by_year.parquet`.

In [44]:
new_water_quality.where(col('DNR_ID_Site_Number').isin(lake_names)).take(5) >> to_pandas

,LAKE_NAME,DNR_ID_Site_Number,Year,Mean_Secchi_Depth_Result,Mean_Phosporus_Result
0,Alimagnet Lake,19002100-01,2006,0.525000,0.154375
1,George Watch Lake,02000500-01,2006,0.728571,0.164286
2,Alimagnet Lake,19002100-01,2014,0.966667,0.082500
3,Eagle Lake,10012100-01,2004,1.036364,0.194818
4,Eagle Lake,10012100-01,2011,0.525000,0.197571


In [45]:
# Your code here
(new_water_quality
 .where(col('DNR_ID_Site_Number').isin(lake_names))
 .write
 .partitionBy('Year')
 .mode('overwrite')
 .parquet(f'./data/water_quality_by_year.parquet')
)

## Deliverables.

Be sure to push your final notebook, as well as the final water quality file to GitHub.